In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
# Get the data
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [3]:
tokenizer = info.features['text'].encoder

In [4]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
train_shapes = tf.compat.v1.data.get_output_shapes(train_dataset)
print(train_shapes)
test_shapes = tf.compat.v1.data.get_output_shapes(test_dataset)
print(test_shapes)
# train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_shapes)
# test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_shapes)

(TensorShape([None]), TensorShape([]))
(TensorShape([None]), TensorShape([]))


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False):
    shape = tf.compat.v1.data.get_output_shapes(ds)
    ds.cache()
    
    if shuffle:
        ds = ds.shuffle(BUFFER_SIZE)
        
    ds = ds.padded_batch(BATCH_SIZE, shape)
    
    return ds.prefetch(buffer_size=AUTOTUNE)
train_dataset = prepare(train_dataset, shuffle=True)
test_dataset = prepare(test_dataset)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
NUM_EPOCHS = 10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/10
391/391 [==============================] - 187s 459ms/step - loss: 0.6095 - accuracy: 0.6405 - val_loss: 0.5283 - val_accuracy: 0.7338
Epoch 2/10
391/391 [==============================] - 174s 446ms/step - loss: 0.4591 - accuracy: 0.7883 - val_loss: 0.6999 - val_accuracy: 0.6329
Epoch 3/10
391/391 [==============================] - 174s 445ms/step - loss: 0.4868 - accuracy: 0.7712 - val_loss: 0.4618 - val_accuracy: 0.7920
Epoch 4/10
391/391 [==============================] - 175s 447ms/step - loss: 0.3287 - accuracy: 0.8680 - val_loss: 0.3730 - val_accuracy: 0.8433
Epoch 5/10
391/391 [==============================] - 173s 443ms/step - loss: 0.2685 - accuracy: 0.8964 - val_loss: 0.3969 - val_accuracy: 0.8123
Epoch 6/10
391/391 [==============================] - 173s 444ms/step - loss: 0.2298 - accuracy: 0.9144 - val_loss: 0.3919 - val_accuracy: 0.8367
Epoch 7/10
391/391 [==============================] - 173s 443ms/step - loss: 0.1709 - accuracy: 0.9405 - val_loss: 0.3495 -

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')